<a href="https://colab.research.google.com/github/Devsachin2003/Prognosticating-Different-stages-of-Goiter-using-ML-algorithms-/blob/main/Random_forest_classifier_for_Goiter_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thyroid Disease Detection
This notebook demonstrates the process of loading, preprocessing, and training a Random Forest classifier to detect goitre using thyroid disease data.

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import drive

## 1. Load Dataset
Mount Google Drive and load the `sick_csv.csv` file. Please ensure the file exists in the specified path.

In [6]:
from google.colab import drive
import pandas as pd
import os

# 1. Mount the drive to the standard folder (Do not change this path)
drive.mount('/content/drive')

# 2. Define the path to your specific file
# The path typically starts with /content/drive/MyDrive/ followed by your file name
file_path = '/content/drive/MyDrive/Copy of sick_csv.csv'

# 3. Load the dataset
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    display(df.head())
else:
    print(f"File not found at: {file_path}")
    print("Please check the file name or use the file browser on the left to copy the path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded successfully.


,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41.0,F,f,f,f,f,f,f,f,f,...,t,125.0,t,1.14,t,109.0,f,NaN,SVHC,negative
1,23.0,F,f,f,f,f,f,f,f,f,...,t,102.0,f,NaN,f,NaN,f,NaN,other,negative
2,46.0,M,f,f,f,f,f,f,f,f,...,t,109.0,t,0.91,t,120.0,f,NaN,other,negative
3,70.0,F,t,f,f,f,f,f,f,f,...,t,175.0,f,NaN,f,NaN,f,NaN,other,negative
4,70.0,F,f,f,f,f,f,f,f,f,...,t,61.0,t,0.87,t,70.0,f,NaN,SVI,negative


## 2. Data Preprocessing
Convert categorical variables to numerical formats and handle missing values.

In [7]:
if 'df' in locals():
    # Replace 'f' with 0 and 't' with 1 in 'goitre' column
    df['goitre'] = df['goitre'].map({'f': 0, 't': 1})

    # Convert binary categorical columns to numerical
    binary_categorical_cols = ['sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication',
                               'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment', 'query_hypothyroid',
                               'query_hyperthyroid', 'lithium', 'tumor', 'hypopituitary', 'psych',
                               'TSH_measured', 'T3_measured', 'TT4_measured', 'T4U_measured', 'FTI_measured',
                               'TBG_measured']

    for col in binary_categorical_cols:
        df[col] = df[col].map({'f': 0, 't': 1})

    # One-hot encode multi-category columns
    multi_categorical_cols = ['referral_source', 'Class']
    df = pd.get_dummies(df, columns=multi_categorical_cols)

    # Define features and target variable
    X = df.drop("goitre", axis=1)  # Features
    y = df["goitre"]  # Target

    # Impute missing values using SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    print("Preprocessing complete.")

Preprocessing complete.


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sex' 'TBG']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


## 3. Model Training
Split the data into training and testing sets, then train a Random Forest Classifier.

In [8]:
if 'X_imputed' in locals() and 'y' in locals():
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.015, random_state=10)

    # Train the Random Forest classifier on imputed data
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=11)
    rf_classifier.fit(X_train, y_train)
    print("Model trained successfully.")

Model trained successfully.


## 4. Evaluation
Evaluate the model's performance on the test set.

In [9]:
if 'rf_classifier' in locals() and 'X_test' in locals():
    # Make predictions on the test set
    y_pred = rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy_randomforest = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy_randomforest)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9649122807017544

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        55
           1       0.00      0.00      0.00         2

    accuracy                           0.96        57
   macro avg       0.48      0.50      0.49        57
weighted avg       0.93      0.96      0.95        57



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
